## AutoGen with Google Cloud Storage Integration
### Overview
This Jupyter notebook demonstrates how to integrate AutoGen (multi-agent conversation framework) with Google Cloud Storage (GCS) for automated file management tasks.

## What is AutoGen?
AutoGen is Microsoft's cutting-edge framework for creating multi-agent conversational AI systems. It enables multiple AI agents to collaborate, debate, and solve complex problems through structured conversations.
Key AutoGen Features:

* Multi-Agent Conversations: Multiple AI agents can talk to each other
* Role Specialization: Each agent can have specific roles and capabilities
* Function Calling: Agents can execute real-world functions and tools
* Conversation Flow Control: Sophisticated conversation management
* Human-in-the-Loop: Optional human oversight and intervention

## What is Google Cloud Storage (GCS)?
Google Cloud Storage is Google's object storage service for storing and accessing data on Google Cloud Platform. It's designed for storing any amount of data and retrieving it as often as needed.
Key GCS Features:

* Scalable Storage: Store from gigabytes to petabytes
* Global Accessibility: Access data from anywhere in the world
* Security: Enterprise-grade security and access controls
* Cost-Effective: Multiple storage classes for different use cases
* Integration: Works seamlessly with other Google Cloud services

Dependencies and Installation

In [ ]:
!pip install autogen pyautogen google-generativeai openai python-dotenv langchain_google_genai ag2[gemini] vertexai

## Required Imports

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
import autogen
from autogen import AssistantAgent, UserProxyAgent
from google.cloud import storage

## Configuration Setup
1. Google Cloud Authentication

In [ ]:
api_key = "your_Cloud_Authentication_json_file"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = api_key

In [ ]:
# Test credentials
try:
    client = storage.Client(project="Gen-ai") # project Name Gen-ai
    print(f"Successfully connected to GCP project: {client.project}")
except Exception as e:
    print(f"Failed to connect to GCP: {str(e)}")
    exit()

Successfully connected to GCP project: Gen-ai


## 2. OpenAI API Configuration

In [ ]:
# Your exact setup
os.environ["OPENAI_API_KEY"] = "Your_API_Key"


In [ ]:
config_list = [
    {
        "model": "gpt-4o",
        "api_key": os.getenv("OPENAI_API_KEY")
    }
]

## 3. Base Directory Setup

In [ ]:
from pathlib import Path
# Define the base directory using pathlib (no escape sequence issues)
BASE_DIR = Path("/content")

In [ ]:
# CORRECT: Set BASE_DIR to the directory, not the file
BASE_DIR = Path("/content")  # This is the directory in Colab

# Alternative: You can also use
# BASE_DIR = Path("/content/")

print(f"Base directory: {BASE_DIR}")
print(f"Base directory exists: {BASE_DIR.exists()}")

# List files in the directory
if BASE_DIR.exists():
    files = [f.name for f in BASE_DIR.iterdir()]
    print(f"Files in {BASE_DIR}: {files}")

Base directory: /content
Base directory exists: True
Files in /content: ['.config', 'temp_eer_2.pdf', 'gen-ai-462005-7ec6dfae325a.json', 'sample_data']


## Core Functions
1. GCS Bucket Listing

In [ ]:
print("=== Step 1: Adding Simple GCP Storage Tool ===")

# Simple GCP Storage function (tool)
def list_gcs_buckets():
    """
    Try to list buckets, but if permission denied, try to access the specific bucket instead
    """
    try:
        client = storage.Client(project="Gen-ai")
        buckets = list(client.list_buckets())
        bucket_names = [bucket.name for bucket in buckets]
        return f"Found {len(bucket_names)} buckets: {', '.join(bucket_names)}"
    except Exception as e:
        # If listing fails due to permissions, try to access the specific bucket
        try:
            client = storage.Client(project="Gen-ai") # GCP Project name
            bucket = client.get_bucket("bucket_demo7") ## use your bucket name create on GCP
            return f"Cannot list all buckets (permission denied), but can access bucket: {bucket.name}"
        except Exception as bucket_error:
            return f"Error accessing GCS: {str(e)}. Also cannot access bucket_demo7: {str(bucket_error)}"

=== Step 1: Adding Simple GCP Storage Tool ===


## 2. File Upload with Debug Information

In [ ]:
def upload_to_gcs_debug(file_name):
    """
    Upload file with debugging - handles the correct filename
    """
    try:
        # Handle common filename variations
        possible_names = [
            file_name,
            file_name.replace('_', ' '),
            file_name.replace(' ', '_'),
            file_name.replace('eer2', 'eer_2'),
            file_name.replace('eer_2', 'eer2')
        ]

        file_path = None
        actual_filename = None

        # Find the actual file
        for name in possible_names:
            test_path = BASE_DIR / name
            if test_path.exists():
                file_path = test_path
                actual_filename = name
                break

        if file_path is None:
            # List all PDF files for reference
            if BASE_DIR.exists():
                pdf_files = [f.name for f in BASE_DIR.iterdir() if f.suffix.lower() == '.pdf']
                return f"❌ File '{file_name}' not found. Available PDF files: {pdf_files}"
            else:
                return f"❌ Directory '{BASE_DIR}' does not exist"

        print(f"Debug: Found file at: {file_path}")
        print(f"Debug: Actual filename: {actual_filename}")
        print(f"Debug: File size: {file_path.stat().st_size} bytes")

        client = storage.Client(project="Gen-ai") ## use your project name
        print("Debug: GCP client created successfully")

        bucket = client.get_bucket("bucket_demo7") ## use your bucket name create on GCP
        print(f"Debug: Bucket retrieved successfully: {bucket.name}")

        # Use the original requested filename for the blob name
        blob = bucket.blob(file_name)
        print(f"Debug: Blob object created: {blob.name}")

        blob.upload_from_filename(str(file_path)) ## use your file path
        print("Debug: Upload completed")

        return f"✅ Successfully uploaded '{actual_filename}' as '{file_name}' to bucket 'bucket_demo7'"
    except Exception as e:
        return f"❌ Error uploading to GCS: {str(e)}"

## 3. Permission Checking

In [ ]:
def check_bucket_permissions():
    """
    Check what permissions we have on the bucket
    """
    try:
        client = storage.Client(project="Gen-ai")
        bucket = client.get_bucket("bucket_demo7")

        # Test various permissions
        permissions_to_test = [
            'storage.objects.create',
            'storage.objects.get',
            'storage.objects.list',
            'storage.buckets.get'
        ]

        allowed_permissions = bucket.test_iam_permissions(permissions_to_test)
        return f"Bucket permissions: {allowed_permissions}"
    except Exception as e:
        return f"Error checking permissions: {str(e)}"

## AutoGen Agent Configuration
1. GCP Assistant Agent

In [ ]:
# Create an agent that can use GCP tools
gcp_assistant = AssistantAgent(
    name="GCPHelper",
    system_message="""
You are a helpful assistant that can work with Google Cloud Storage.

You have access to these functions:
1. check_bucket_permissions() - Check what permissions we have
2. list_gcs_buckets() - Lists buckets (may have permission issues)
3. upload_to_gcs_debug(file_name) - Uploads a file with detailed debugging

When uploading files, the function will automatically handle filename variations
(like temp_eer2.pdf vs temp_eer_2.pdf).
""",
    llm_config={
        "config_list": config_list,
        "functions": [
            {
                "name": "check_bucket_permissions",
                "description": "Check bucket permissions",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            },
            {
                "name": "list_gcs_buckets",
                "description": "List buckets or access specific bucket",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            },
            {
                "name": "upload_to_gcs_debug",
                "description": "Upload a file to GCS bucket with debugging",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "file_name": {
                            "type": "string",
                            "description": "Name of the file to upload"
                        }
                    },
                    "required": ["file_name"]
                }
            }
        ]
    }
)

## 2. User Proxy Agent

In [ ]:
# Create user proxy
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    code_execution_config=False,
    function_map={
        "check_bucket_permissions": check_bucket_permissions,
        "list_gcs_buckets": list_gcs_buckets,
        "upload_to_gcs_debug": upload_to_gcs_debug
    }
)

## Usage Example
Running the Conversation

In [ ]:
print("=== Step 2: Testing GCP Tool Integration ===")

# Test the GCP tool integration
try:
    conversation = user_proxy.initiate_chat(
        gcp_assistant,
        message="Please check bucket permissions, then try to upload the temp_eer_2.pdf file."
    )
except Exception as e:
    print(f"Error in AutoGen conversation: {str(e)}")

=== Step 2: Testing GCP Tool Integration ===
User (to GCPHelper):

Please check bucket permissions, then try to upload the temp_eer_2.pdf file.

--------------------------------------------------------------------------------
GCPHelper (to User):

***** Suggested function call: check_bucket_permissions *****
Arguments: 
{}
*************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION check_bucket_permissions...
Call ID: None
Input arguments: {}

>>>>>>>> EXECUTED FUNCTION check_bucket_permissions...
Call ID: None
Input arguments: {}
Output:
Bucket permissions: ['storage.buckets.get', 'storage.objects.create', 'storage.objects.get', 'storage.objects.list']
User (to GCPHelper):

***** Response from calling function (check_bucket_permissions) *****
Bucket permissions: ['storage.buckets.get', 'storage.objects.create', 'storage.objects.get', 'storage.objects.list']
****************